In [1]:
pip install flask

In [2]:
pip install pyngrok

In [3]:
pip install flask-ngrok

# Create Flask web application using ngrok
เราจะใช้ ngrok ในการ delopy model ขึ้นดังนั้น ก่อนจะทำการ deploy จะต้องมีการติดตั้งและการ authen  token  ก่อน ให้สมัครสมาชิกและทำการ generate token ขึ้นมา
ไปที่ https://dashboard.ngrok.com/login

In [4]:
!ngrok authtoken 2dZuDxNMKM3HEXespBLXydGkr4R_4BNdG6AD8KQCSdGKPbKDx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
from flask import Flask
from pyngrok import ngrok

app = Flask(__name__)


# Start ngrok to expose your Flask app on port 5000
ngrok_tunnel = ngrok.connect(5000)

# Print the public URL
print('Public URL:', ngrok_tunnel.public_url)

@app.route('/')
def hello():
    return 'Hello, Hello World!'

if __name__ == '__main__':
    # Run the Flask app
    app.run()

Public URL: https://65c9-35-237-197-162.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Mar/2024 07:23:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Mar/2024 07:23:12] "GET /favicon.ico HTTP/1.1" 404 -


# Load Model

In [6]:
from google.colab import drive
drive.mount('/content/drive') #ระวังเครื่องหมาย ‘ กับ ' ต่างกันนะคะ

Mounted at /content/drive


In [7]:
cd /content/drive/MyDrive/Colab Notebooks/web_iris

/content/drive/MyDrive/Colab Notebooks/web_iris


In [8]:
ls

'Deploy Model on  web using Flask and Ngrok.ipynb'   iris_model.h5      iris_unseen.csv
 index.html                                          iris_model.pkl     static/
 iris.csv                                            iris_unseen.arff   templates/


In [9]:
from numpy import loadtxt
from keras.models import load_model
#load model
model = load_model('iris_model.h5')
#summaize model
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 10)                50        
                                                                 
 dense_34 (Dense)            (None, 8)                 88        
                                                                 
 dense_35 (Dense)            (None, 6)                 54        
                                                                 
 dense_36 (Dense)            (None, 3)                 21        
                                                                 
Total params: 213 (852.00 Byte)
Trainable params: 213 (852.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Loading unseen data

In [10]:
import numpy as np
import pandas as pd

udata = pd.read_csv('iris_unseen.csv')
Xnew= udata.iloc[:,0:4]

Xnew

,sepal_length,sepal_width,petal_length,petal_width
0,3.5,4.5,2.1,1.3
1,4.2,2.1,1.6,2.6
2,5.3,3.2,6.5,1.2
3,4.3,2.8,2.5,0.3
4,5.9,3.0,5.1,1.8


# Predict unseen using iris_model.h5

In [11]:
yacc= model.predict(Xnew)
yacc

1/1 [==============================] - 0s 199ms/step


array([[2.6549461e-01, 5.0633585e-01, 2.2816953e-01],
       [7.5801200e-01, 1.6245849e-01, 7.9529501e-02],
       [4.5819458e-04, 1.5807827e-01, 8.4146351e-01],
       [3.7966010e-01, 4.2055047e-01, 1.9978952e-01],
       [5.4770284e-03, 3.0864683e-01, 6.8587613e-01]], dtype=float32)

In [12]:
classes_x=np.argmax(yacc,axis=-1) #majority vote from the prediction

for i in range(len(classes_x)):
  print("The label of unsenn data is ", yacc[i], '\t', classes_x[i])

The label of unsenn data is  [0.2654946  0.50633585 0.22816953] 	 1
The label of unsenn data is  [0.758012  0.1624585 0.0795295] 	 0
The label of unsenn data is  [4.5819458e-04 1.5807827e-01 8.4146351e-01] 	 2
The label of unsenn data is  [0.3796601  0.42055047 0.19978952] 	 1
The label of unsenn data is  [0.00547703 0.30864683 0.68587613] 	 2


# Deploying your model on web application


In [13]:
from flask import Flask, request, render_template
import numpy as np
from keras.models import load_model
from pyngrok import ngrok

app = Flask(__name__)

model = load_model('iris_model.h5')

# Start ngrok to expose your Flask app on port 5000
ngrok_tunnel = ngrok.connect(5000)

# Print the public URL
print('Public URL:', ngrok_tunnel.public_url)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=["POST"])

def predict():
    if request.method == 'POST':
        petal_length = request.form['petal_length']
        sepal_length = request.form['sepal_length']
        petal_width = request.form['petal_width']
        sepal_width = request.form['sepal_width']

        sample_data = [sepal_length, sepal_width, petal_length, petal_width]
        clean_data = [float(i) for i in sample_data]

        final_input = np.array(clean_data).reshape(1,-1)
        print(final_input)

        prediction = model.predict(final_input)
        class_unseen=np.argmax(prediction,axis=-1) #majority vote from the prediction
        print("predictedclass ",class_unseen)

    return render_template('index.html',result_prediction=class_unseen)

if __name__ == '__main__':
    # Run the Flask app
    app.run()

Public URL: https://352b-35-237-197-162.ngrok-free.app
Public URL: https://352b-35-237-197-162.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Mar/2024 07:25:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Mar/2024 07:25:55] "GET /favicon.ico HTTP/1.1" 404 -
